# Import Libraries

In [1]:
import pandas as pd
import numpy as np

/Users/azamkhan/anaconda3/envs/nasa/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Import Data and Create Dataset

In [2]:
remaining_input_data = pd.read_csv("data/Data NO3.csv")
water_quality_output_data = pd.read_excel("data/Water Quality Outlet.xlsx")
water_quality_output_data.drop(columns=["Total Phosphorus DlyLd(kg)"], inplace=True)
water_quality_data_post_1993 = water_quality_output_data[water_quality_output_data["DATE"] >= "1993-01-01"]
calibrated_streamflow_df = pd.read_excel("data/Calibrated Streamflow data.xlsx")
calibrated_streamflow_df.rename(columns={"Simulated": "Calibrated_SWAT_Streamflow"}, inplace=True)
calibrated_snowmelt_df = pd.read_csv("data/SWAT_SnowMelt_Historical.csv")
calibrated_snowmelt_df['Date'] = pd.to_datetime(calibrated_snowmelt_df['Date'])

In [3]:
sediments_calibrated_water_quality_df = pd.read_excel(
    "data/CalibratedWaterQuality.xlsx", sheet_name="Sediments"
)
sediments_calibrated_water_quality_df.rename(
    columns={"Simulated": "Calibrated_SWAT_Sediments DlyLd(kg*1000)"}, inplace=True
)
sediments_calibrated_water_quality_df.dropna(inplace=True)

nitrate_calibrated_water_quality_df = pd.read_excel(
    "data/CalibratedWaterQuality.xlsx", sheet_name="Nitrate"
)
nitrate_calibrated_water_quality_df.rename(
    columns={"Simulated": "Calibrated_SWAT_Nitrate DlyLd(kg)"}, inplace=True
)
phosphate_calibrated_water_quality_df = pd.read_excel(
    "data/CalibratedWaterQuality.xlsx", sheet_name="Phosphate"
)
phosphate_calibrated_water_quality_df.rename(
    columns={"Simulated": "Calibrated_SWAT_Phosphate DlyLd(kg)"}, inplace=True
)

calibrated_streamflow_df = pd.read_excel("data/Calibrated Streamflow data.xlsx")
calibrated_streamflow_df.rename(
    columns={"Simulated": "Calibrated_SWAT_Streamflow"}, inplace=True
)

calibrated_snowmelt_df = pd.read_csv("data/SWAT_SnowMelt_Historical.csv")
calibrated_snowmelt_df["Date"] = pd.to_datetime(calibrated_snowmelt_df["Date"])

swat_sediments_df = pd.merge(
    calibrated_streamflow_df, sediments_calibrated_water_quality_df, on="Date"
)
swat_sediments_df = pd.merge(swat_sediments_df, calibrated_snowmelt_df, on="Date")

swat_nitrate_df = pd.merge(
    calibrated_streamflow_df, nitrate_calibrated_water_quality_df, on="Date"
)
swat_nitrate_df = pd.merge(swat_nitrate_df, calibrated_snowmelt_df, on="Date")

swat_phosphate_df = pd.merge(
    calibrated_streamflow_df, phosphate_calibrated_water_quality_df, on="Date"
)
swat_phosphate_df = pd.merge(swat_phosphate_df, calibrated_snowmelt_df, on="Date")

# calibrated_water_quality_df = pd.merge(
#     calibrated_water_quality_df, phosphate_calibrated_water_quality_df, on="Date"
# )

# calibrated_swat_simulations_df = pd.merge(
#     calibrated_streamflow_df, calibrated_water_quality_df, on="Date"
# )


swat_simulations_dfs = {
    "Sediments DlyLd(kg*1000)": swat_sediments_df,
    "Nitrate DlyLd(kg)": swat_nitrate_df,
    "Phosphate DlyLd(kg)": swat_phosphate_df,
}


In [4]:
swat_simulations_dfs['Sediments DlyLd(kg*1000)']

,Date,Calibrated_SWAT_Streamflow,Calibrated_SWAT_Sediments DlyLd(kg*1000),Calibrated_SWAT_Snowmelt
0,1993-01-01,36.110,72.0400,0.029398
1,1993-01-02,36.200,44.4200,0.000000
2,1993-01-03,33.750,26.6100,0.000000
3,1993-01-04,34.430,32.7800,3.790000
4,1993-01-05,46.330,43.5200,1.194448
...,...,...,...,...
6472,2010-09-26,1.093,0.8149,0.000000
6473,2010-09-27,2.336,2.7360,0.000000
6474,2010-09-28,3.154,3.8130,0.000000
6475,2010-09-29,4.023,5.3260,0.000000


In [5]:
from sklearn.metrics import r2_score, mean_absolute_error, median_absolute_error

from scipy.stats import pearsonr

targets = [
    "Phosphate DlyLd(kg)",
    "Nitrate DlyLd(kg)",
    "Sediments DlyLd(kg*1000)",
]

def r_squared(observed, simulated):
    return pearsonr(observed, simulated)[0] ** 2

monthly_aggregate_swat_metrics = {}
swat_dfs = {}
for target in targets:
    date_aligned_df = pd.merge(
        water_quality_data_post_1993[["DATE", target]],
        swat_simulations_dfs[target],
        left_on="DATE",
        right_on="Date",
        how="right",
    ).drop(columns=["DATE"])
    # if target == "Sediments DlyLd(kg*1000)":
    #     date_aligned_df = date_aligned_df[(date_aligned_df["Date"] < "2007-01-01")]

    df = pd.DataFrame(
        {
            "date": date_aligned_df["Date"],
            "observed": date_aligned_df[target],
            "simulated": date_aligned_df[f"Calibrated_SWAT_{target}"],
        }
    )
    # Ensure the Date column is in datetime format
    df["date"] = pd.to_datetime(df["date"])
    df["month"] = df["date"].dt.to_period("M")

    monthly_df = (
        df.groupby("month")
        .agg(observed=("observed", "sum"), simulated=("simulated", "sum"))
        .reset_index()
    )

    swat_dfs[target] = monthly_df
    monthly_r2 = r2_score(monthly_df["observed"], monthly_df["simulated"])
    monthly_correlation = r_squared(monthly_df["observed"], monthly_df["simulated"])
    monthly_medae = median_absolute_error(
        monthly_df["observed"], monthly_df["simulated"]
    )
    # monthly_nse = nse_score(monthly_df["observed"], monthly_df["simulated"])
    # monthly_pbias = pbias_score(monthly_df["observed"], monthly_df["simulated"])

    monthly_aggregate_swat_metrics[target] = {
        "R2": monthly_r2,
        "Correlation": monthly_correlation,
        # "MEDAE": monthly_medae,
        # "NSE": monthly_nse,
        # "PBIAS": monthly_pbias,
    }

monthly_aggregate_swat_metrics_df = pd.DataFrame(monthly_aggregate_swat_metrics).T
monthly_aggregate_swat_metrics_df


,R2,Correlation
Phosphate DlyLd(kg),0.555361,0.560880
Nitrate DlyLd(kg),0.429960,0.524988
Sediments DlyLd(kg*1000),0.225880,0.376002


In [6]:
monthly_df


,month,observed,simulated
0,1993-01,727.587922,991.4730
1,1993-02,23.161933,35.7240
2,1993-03,12716.462017,2895.0531
3,1993-04,7869.031317,2738.4500
4,1993-05,94.675734,1009.5020
...,...,...,...
208,2010-05,243.455462,1195.5400
209,2010-06,106.012377,911.1360
210,2010-07,52.920583,838.8940
211,2010-08,315.021770,1304.6350


In [7]:
sediments_calibrated_water_quality_df =pd.read_excel("data/CalibratedWaterQuality.xlsx", sheet_name="Sediments")
sediments_calibrated_water_quality_df.rename(columns={"Simulated": "Calibrated_SWAT_Sediments DlyLd(kg*1000)"}, inplace=True)
nitrate_calibrated_water_quality_df =pd.read_excel("data/CalibratedWaterQuality.xlsx", sheet_name="Nitrate")
nitrate_calibrated_water_quality_df.rename(columns={"Simulated": "Calibrated_SWAT_Nitrate DlyLd(kg)"}, inplace=True)
phosphate_calibrated_water_quality_df =pd.read_excel("data/CalibratedWaterQuality.xlsx", sheet_name="Phosphate")
phosphate_calibrated_water_quality_df.rename(columns={"Simulated": "Calibrated_SWAT_Phosphate DlyLd(kg)"}, inplace=True)

calibrated_water_quality_df = pd.merge(
    sediments_calibrated_water_quality_df, nitrate_calibrated_water_quality_df, on="Date"
)

calibrated_water_quality_df = pd.merge(
    calibrated_water_quality_df, phosphate_calibrated_water_quality_df, on="Date"
)

calibrated_swat_simulations_df = pd.merge(
    calibrated_streamflow_df, calibrated_water_quality_df, on="Date"
)

calibrated_swat_simulations_df = pd.merge(
    calibrated_swat_simulations_df, calibrated_snowmelt_df, on="Date"
)


calibrated_swat_simulations_df

,Date,Calibrated_SWAT_Streamflow,Calibrated_SWAT_Sediments DlyLd(kg*1000),Calibrated_SWAT_Nitrate DlyLd(kg),Calibrated_SWAT_Phosphate DlyLd(kg),Calibrated_SWAT_Snowmelt
0,1993-01-01,36.110,72.0400,874.90,49.8900,0.029398
1,1993-01-02,36.200,44.4200,232.20,17.2600,0.000000
2,1993-01-03,33.750,26.6100,1133.00,39.9900,0.000000
3,1993-01-04,34.430,32.7800,1213.00,72.1200,3.790000
4,1993-01-05,46.330,43.5200,1135.00,116.7000,1.194448
...,...,...,...,...,...,...
6477,2010-09-26,1.093,0.8149,17.45,0.2327,0.000000
6478,2010-09-27,2.336,2.7360,66.74,11.9800,0.000000
6479,2010-09-28,3.154,3.8130,88.67,16.6600,0.000000
6480,2010-09-29,4.023,5.3260,150.10,21.2600,0.000000


In [8]:
climate_data_df = remaining_input_data.drop(columns=["NO3", "SWATNO3", "Streamflow"])


start_year = 1993
end_year = 2010
total_years = end_year - start_year + 1

# Calculate the corresponding year for each day of the year
climate_data_df["year"] = (climate_data_df.index // 365) + start_year

# Ensure the year does not exceed the end year
climate_data_df = climate_data_df[climate_data_df["year"] <= end_year]

climate_data_df['date'] = pd.to_datetime(climate_data_df['year'].astype(str) + '-' + (climate_data_df.index % 365 + 1).astype(str), format='%Y-%j')

climate_data_df.drop(columns=["year"], inplace=True)

In [9]:
climate_data_df

,DOY,Precipitation,Evapotranspiration,MaxT,MinT,date
0,1,0.330769,0.517,2.445385,-10.748462,1993-01-01
1,2,0.101538,0.402,-3.736154,-10.940769,1993-01-02
2,3,1.456923,0.782,1.354615,-10.342308,1993-01-03
3,4,5.001538,0.378,8.511538,-1.980769,1993-01-04
4,5,12.960000,0.412,10.614615,-5.325385,1993-01-05
...,...,...,...,...,...,...
6477,269,0.176923,0.786,18.566923,7.568462,2010-09-30
6478,270,10.595385,1.170,19.443846,10.580769,2010-10-01
6479,271,6.947692,2.150,22.430000,9.900769,2010-10-02
6480,272,3.652308,2.320,21.366923,9.922308,2010-10-03


In [10]:
#Create Training Set

training_data = pd.merge(climate_data_df, calibrated_swat_simulations_df, left_on="date", right_on="Date")
training_data.drop(columns=["date"], inplace=True)
# dataset.to_csv("data/cal_swat_climate_dataset.csv", index=False)

dataset = pd.merge(training_data, water_quality_data_post_1993, left_on="Date", right_on="DATE")
dataset.drop(columns=[ "DATE"], inplace=True)

In [11]:
dataset

,DOY,Precipitation,Evapotranspiration,MaxT,MinT,Date,Calibrated_SWAT_Streamflow,Calibrated_SWAT_Sediments DlyLd(kg*1000),Calibrated_SWAT_Nitrate DlyLd(kg),Calibrated_SWAT_Phosphate DlyLd(kg),Calibrated_SWAT_Snowmelt,Phosphate DlyLd(kg),Sediments DlyLd(kg*1000),Nitrate DlyLd(kg)
0,1,0.330769,0.517,2.445385,-10.748462,1993-01-01,36.110,72.0400,874.90,49.8900,0.029398,246.66200,160.755731,4579.05160
1,2,0.101538,0.402,-3.736154,-10.940769,1993-01-02,36.200,44.4200,232.20,17.2600,0.000000,147.97600,40.571564,3446.03880
2,3,1.456923,0.782,1.354615,-10.342308,1993-01-03,33.750,26.6100,1133.00,39.9900,0.000000,86.33700,22.013847,2764.18320
3,4,5.001538,0.378,8.511538,-1.980769,1993-01-04,34.430,32.7800,1213.00,72.1200,3.790000,79.86040,36.838848,2182.01000
4,5,12.960000,0.412,10.614615,-5.325385,1993-01-05,46.330,43.5200,1135.00,116.7000,1.194448,212.74200,218.350895,3680.86060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6473,265,1.153846,1.560,25.570000,9.843846,2010-09-26,1.093,0.8149,17.45,0.2327,0.000000,1.52640,3.006372,42.99360
6474,266,0.000000,1.120,23.216154,10.510000,2010-09-27,2.336,2.7360,66.74,11.9800,0.000000,2.24720,4.852436,51.18740
6475,267,0.000000,1.980,30.226923,16.171538,2010-09-28,3.154,3.8130,88.67,16.6600,0.000000,2.81960,5.639539,139.32640
6476,268,0.000000,1.030,23.670000,7.502308,2010-09-29,4.023,5.3260,150.10,21.2600,0.000000,9.05240,15.536208,889.93360


In [12]:
dataset.describe()

,DOY,Precipitation,Evapotranspiration,MaxT,MinT,Date,Calibrated_SWAT_Streamflow,Calibrated_SWAT_Sediments DlyLd(kg*1000),Calibrated_SWAT_Nitrate DlyLd(kg),Calibrated_SWAT_Phosphate DlyLd(kg),Calibrated_SWAT_Snowmelt,Phosphate DlyLd(kg),Sediments DlyLd(kg*1000),Nitrate DlyLd(kg)
count,6478.000000,6478.000000,6478.000000,6478.000000,6478.000000,6478,6478.000000,6473.000000,6478.000000,6478.000000,6478.000000,6478.000000,6473.000000,6478.000000
mean,181.119790,3.289554,1.782721,13.077131,0.976026,2001-11-15 05:54:33.232479232,18.022557,42.951815,647.953376,38.042383,0.547622,35.087614,59.810277,881.826510
min,1.000000,0.000000,0.012400,-19.305385,-28.971538,1993-01-01 00:00:00,0.009484,0.003361,0.241100,0.004685,0.000000,0.201400,0.045092,1.038800
25%,90.250000,0.000000,0.629250,3.821538,-5.860000,1997-06-09 06:00:00,4.230750,6.689000,60.412500,2.263750,0.000000,5.822050,1.306217,190.028850
50%,180.000000,0.344615,1.550000,14.194615,1.197692,2001-11-15 12:00:00,9.399000,25.100000,209.000000,10.335000,0.000000,12.873700,4.112715,473.717180
75%,270.000000,3.092308,2.720000,22.618846,9.151923,2006-04-23 18:00:00,24.237500,61.370000,657.300000,40.825000,0.000000,29.226850,13.462912,1055.495000
max,366.000000,113.849231,6.070000,33.268462,20.010000,2010-09-30 00:00:00,359.800000,1307.000000,24260.000000,1065.000000,28.030769,3936.795480,15298.070266,45621.128000
std,104.876329,7.113935,1.274348,10.728194,9.652153,NaN,24.463606,65.085344,1385.750737,77.444728,2.406185,120.809620,419.853641,1475.307285


In [13]:
dataset.to_csv("data/cal_swat_climate_dataset.csv", index=False)

# Feature Engineering

In [14]:
dataset['Date'] = pd.to_datetime(dataset['Date'])
# dataset['year'] = dataset['Date'].dt.year
dataset['month'] = dataset['Date'].dt.month

# Create cyclic features for DOY (day of the year)
dataset['DOY_cos'] = np.cos(2 * np.pi * dataset['DOY'] / 365)
dataset['DOY_sin'] = np.sin(2 * np.pi * dataset['DOY'] / 365)

# Create cyclic features for month
dataset['month_cos'] = np.cos(2 * np.pi * dataset['month'] / 12)
dataset['month_sin'] = np.sin(2 * np.pi * dataset['month'] / 12)

# Calculate temperature range (diurnal temperature range)
dataset['Temp_range'] = dataset['MaxT'] - dataset['MinT']
dataset.drop(['DOY'], axis=1, inplace=True)
dataset.drop(['month'], axis=1, inplace=True)
# Display the updated datasetframe with new features
dataset.rename(columns={"Calibrated_SWAT_Snowmelt": "sim_snowmelt"}, inplace=True)

dataset.head()

,Precipitation,Evapotranspiration,MaxT,MinT,Date,Calibrated_SWAT_Streamflow,Calibrated_SWAT_Sediments DlyLd(kg*1000),Calibrated_SWAT_Nitrate DlyLd(kg),Calibrated_SWAT_Phosphate DlyLd(kg),sim_snowmelt,Phosphate DlyLd(kg),Sediments DlyLd(kg*1000),Nitrate DlyLd(kg),DOY_cos,DOY_sin,month_cos,month_sin,Temp_range
0,0.330769,0.517,2.445385,-10.748462,1993-01-01,36.11,72.04,874.9,49.89,0.029398,246.6620,160.755731,4579.0516,0.999852,0.017213,0.866025,0.5,13.193846
1,0.101538,0.402,-3.736154,-10.940769,1993-01-02,36.20,44.42,232.2,17.26,0.000000,147.9760,40.571564,3446.0388,0.999407,0.034422,0.866025,0.5,7.204615
2,1.456923,0.782,1.354615,-10.342308,1993-01-03,33.75,26.61,1133.0,39.99,0.000000,86.3370,22.013847,2764.1832,0.998667,0.051620,0.866025,0.5,11.696923
3,5.001538,0.378,8.511538,-1.980769,1993-01-04,34.43,32.78,1213.0,72.12,3.790000,79.8604,36.838848,2182.0100,0.997630,0.068802,0.866025,0.5,10.492308
4,12.960000,0.412,10.614615,-5.325385,1993-01-05,46.33,43.52,1135.0,116.70,1.194448,212.7420,218.350895,3680.8606,0.996298,0.085965,0.866025,0.5,15.940000


In [15]:
import pandas as pd


def create_lagged_and_cumulative_features(dataset, lag_days, cumulation_days):
    """
    Creates lagged and cumulative features for specified columns in a Pandas DataFrame.

    Args:
        dataset (pd.DataFrame): The input DataFrame.
        lag_days (list): List of integers representing the lag days.
        cumulation_days (list): List of integers representing the cumulation days.

    Returns:
        pd.DataFrame: The DataFrame with added lagged and cumulative features.
    """

    new_dataset = dataset.copy()

    # Create lagged features
    for lag in lag_days:
        new_dataset[f"Precipitation_lag{lag}"] = new_dataset["Precipitation"].shift(lag)
        new_dataset[f"Evapotranspiration_lag{lag}"] = new_dataset[
            "Evapotranspiration"
        ].shift(lag)
        new_dataset[f"snowmelt_lag{lag}"] = new_dataset["sim_snowmelt"].shift(lag)
        new_dataset[f"MinT_lag{lag}"] = new_dataset["MinT"].shift(lag)
        new_dataset[f"MaxT_lag{lag}"] = new_dataset["MaxT"].shift(lag)

    # Create cumulative features
    for cumulation in cumulation_days:
        new_dataset[f"Precipitation_cum{cumulation}"] = (
            new_dataset["Precipitation"].rolling(cumulation).sum()
        )
        new_dataset[f"Evapotranspiration_cum{cumulation}"] = (
            new_dataset["Evapotranspiration"].rolling(cumulation).sum()
        )
        new_dataset[f"snowmelt_cum{cumulation}"] = (
            new_dataset["sim_snowmelt"].rolling(cumulation).sum()
        )

    # Calculate the maximum lag and cumulation to determine the starting row for slicing.
    max_lag = max(lag_days) if lag_days else 0
    max_cumulation = max(cumulation_days) if cumulation_days else 0
    start_row = max(max_lag, max_cumulation)

    # Slice the DataFrame to remove NaN values from lagging and cumulative features.
    new_dataset = new_dataset[start_row:]

    return new_dataset


In [16]:
dataset_lagged = create_lagged_and_cumulative_features(
    dataset,
    lag_days=[1, 3, 7],
    cumulation_days=[3, 7],
)

In [17]:
datasets = {}

datasets["Nitrate DlyLd(kg)"] = dataset_lagged.drop(
    ["Phosphate DlyLd(kg)", "Sediments DlyLd(kg*1000)"],
    axis=1,
)
datasets["Phosphate DlyLd(kg)"] = dataset_lagged.drop(
    ["Nitrate DlyLd(kg)", "Sediments DlyLd(kg*1000)"], axis=1
)
datasets["Sediments DlyLd(kg*1000)"] = dataset_lagged.drop(
    ["Nitrate DlyLd(kg)", "Phosphate DlyLd(kg)"], axis=1
)
for target in targets:
    datasets[target].dropna(inplace=True)


In [24]:
for target in targets:
    datasets[target].to_csv(
        f"preprocessed_data/{target}.csv",
        index=False,
    )